In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
from tensorflow.keras import Model, layers
import numpy as np
import json
import os

ModuleNotFoundError: No module named 'tensorflow'

In [5]:
import numpy as np
import json
import os
import pandas as pd
import csv
import random

In [2]:
with open("../../data/train_test_spit.json", "r") as f:
    split = json.load(f)

In [3]:
EMBEDDING_FILENAME = os.path.join("..", "..", "data", "wikilinks_train.emb")
embeddings = np.genfromtxt(EMBEDDING_FILENAME, delimiter=',')

In [10]:
int(len(split["train"]["1"]) * 0.02)

142930

In [11]:
tr_1 = random.sample(split["train"]["1"], int(len(split["train"]["1"]) * 0.02))
tr_0 = random.sample(split["train"]["0"], int(len(split["train"]["0"]) * 0.02))

In [17]:
te_1 = random.sample(split["test"]["1"], int(len(split["test"]["1"]) * 0.02))
te_0 = random.sample(split["test"]["0"], int(len(split["test"]["0"]) * 0.02))

In [19]:
with open("train.csv", "w") as f:
    writer = csv.writer(f, delimiter=',')
    for i, j in tr_1:
        writer.writerow(np.concatenate((embeddings[i], embeddings[j], np.array([1])), axis=0))
    print("sem tu")
    for i, j in tr_0:
        writer.writerow(np.concatenate((embeddings[i], embeddings[j], np.array([0])), axis=0))

sem tu


In [20]:
with open("test.csv", "w") as f:
    writer = csv.writer(f, delimiter=',')
    for i, j in te_1:
        writer.writerow(np.concatenate((embeddings[i], embeddings[j], np.array([1])), axis=0))
    print("sem tu")
    for i, j in te_0:
        writer.writerow(np.concatenate((embeddings[i], embeddings[j], np.array([0])), axis=0))

sem tu


In [ ]:
for i, j in split["train"]["0"]:
    train.append(list(np.concatenate((embeddings[i], embeddings[j]), axis=0)))
    train.append(0)

In [ ]:
with open("train.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(train)

In [37]:
columns = list(range(len(embeddings[0])*2))
columns.append("label")

train  = pd.DataFrame(columns=columns)
test  = pd.DataFrame(columns=columns)

In [38]:
train

,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,255,label


In [41]:
for i, j in split["train"]["1"]:
    row = dict(enumerate(np.concatenate((embeddings[i], embeddings[j]), axis=0)))
    row["label"] = 1
    train = pd.concat([train,  pd.DataFrame(row, index=[0])], ignore_index = True, axis = 0)

KeyboardInterrupt: 

In [5]:
def construct_dataset( embeddings, links, class_):
    for i, j in links:
        row = dict(enumerate(np.concatenate((embeddings[i], embeddings[j]), axis=0)))
        row["label"] = class_
        break
    return X, y

In [6]:
X_train, y_train, X_test, y_test = [], [], [], []

X_train, y_train = construct_dataset(X_train, y_train, embeddings, split["train"]["1"], 1)
X_train, y_train = construct_dataset(X_train, y_train, embeddings, split["train"]["0"], 0)

In [7]:
import csv

In [8]:
with open('X_train', 'w') as f:
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerows(X_train)

In [ ]:
num_classes = 1 # Link exists or not

# Training parameters.
learning_rate = 0.001
training_steps = 200
batch_size = 128
display_step = 10

# Network parameters.
conv1_filters = 32 # number of filters for 1st conv layer.
conv2_filters = 64 # number of filters for 2nd conv layer.
fc1_units = 1024 # number of neurons for 1st fully-connected layer.

In [ ]:
# Use tf.data API to shuffle and batch data.
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [ ]:
# Create TF Model.
class ConvNet(Model):
    # Set layers.
    def __init__(self):
        super(ConvNet, self).__init__()
        # Convolution Layer with 32 filters and a kernel size of 5.
        self.conv1 = layers.Conv1D(32, kernel_size=5, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with kernel size of 2 and strides of 2. 
        self.maxpool1 = layers.MaxPool1D(2, strides=2)

        # Convolution Layer with 64 filters and a kernel size of 3.
        self.conv2 = layers.Conv1D(64, kernel_size=3, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with kernel size of 2 and strides of 2. 
        self.maxpool2 = layers.MaxPool1D(2, strides=2)

        # Flatten the data to a 1-D vector for the fully connected layer.
        self.flatten = layers.Flatten()

        # Fully connected layer.
        self.fc1 = layers.Dense(1024)
        # Apply Dropout (if is_training is False, dropout is not applied).
        self.dropout = layers.Dropout(rate=0.5)

        # Output layer, class prediction.
        self.out = layers.Dense(num_classes)

    # Set forward pass.
    def call(self, x, is_training=False):
        x = tf.reshape(x, [-1, 28, 28, 1])
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout(x, training=is_training)
        x = self.out(x)
        if not is_training:
            # tf cross entropy expect logits without softmax, so only
            # apply softmax when not training.
            x = tf.nn.softmax(x)
        return x

In [ ]:
# Build neural network model.
conv_net = ConvNet()

In [ ]:
# Cross-Entropy Loss.
# Note that this will apply 'softmax' to the logits.
def cross_entropy_loss(x, y):
    # Convert labels to int 64 for tf cross-entropy function.
    y = tf.cast(y, tf.int64)
    # Apply softmax to logits and compute cross-entropy.
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    # Average loss across the batch.
    return tf.reduce_mean(loss)

# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.Adam(learning_rate)

In [ ]:
# Optimization process. 
def run_optimization(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        # Forward pass.
        pred = conv_net(x, is_training=True)
        # Compute loss.
        loss = cross_entropy_loss(pred, y)
        
    # Variables to update, i.e. trainable variables.
    trainable_variables = conv_net.trainable_variables

    # Compute gradients.
    gradients = g.gradient(loss, trainable_variables)
    
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, trainable_variables))

In [ ]:
# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_y)
    
    if step % display_step == 0:
        pred = conv_net(batch_x)
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

In [ ]:
# Test model on validation set.
pred = conv_net(X_test)
print("Test Accuracy: %f" % accuracy(pred, y_test))